In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


Suma całkowita

In [ ]:
%%bigquery --project produkcja-instore

WITH Produce AS
 (SELECT 'kale' as item, 23 as purchases, 'vegetable' as category
  UNION ALL SELECT 'orange', 2, 'fruit'
  UNION ALL SELECT 'cabbage', 9, 'vegetable'
  UNION ALL SELECT 'apple', 8, 'fruit'
  UNION ALL SELECT 'leek', 2, 'vegetable'
  UNION ALL SELECT 'lettuce', 10, 'vegetable')
 
select item, purchases, category, sum(purchases) over() total_purchases
from produce

,item,purchases,category,total_purchases
0,kale,23,vegetable,54
1,orange,2,fruit,54
2,cabbage,9,vegetable,54
3,apple,8,fruit,54
4,leek,2,vegetable,54
5,lettuce,10,vegetable,54


Suma częściowa

In [ ]:
%%bigquery --project produkcja-instore

WITH Produce AS
 (SELECT 'kale' as item, 23 as purchases, 'vegetable' as category
  UNION ALL SELECT 'orange', 2, 'fruit'
  UNION ALL SELECT 'cabbage', 9, 'vegetable'
  UNION ALL SELECT 'apple', 8, 'fruit'
  UNION ALL SELECT 'leek', 2, 'vegetable'
  UNION ALL SELECT 'lettuce', 10, 'vegetable')
 
select item, purchases, category, 
sum(purchases) over (partition by category order by purchases
ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) total_purchases
from produce

,item,purchases,category,total_purchases
0,orange,2,fruit,10
1,apple,8,fruit,10
2,leek,2,vegetable,44
3,cabbage,9,vegetable,44
4,lettuce,10,vegetable,44
5,kale,23,vegetable,44


Suma skumulowana - obliczana jest w odniesieniu do kolejności określonej za pomocą Order By


In [ ]:
%%bigquery --project produkcja-instore

WITH Produce AS
 (SELECT 'kale' as item, 23 as purchases, 'vegetable' as category
  UNION ALL SELECT 'orange', 2, 'fruit'
  UNION ALL SELECT 'cabbage', 9, 'vegetable'
  UNION ALL SELECT 'apple', 8, 'fruit'
  UNION ALL SELECT 'leek', 2, 'vegetable'
  UNION ALL SELECT 'lettuce', 10, 'vegetable')
 
select item, purchases, category, 
sum(purchases) over (partition by category order by purchases
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) total_purchases
from produce

,item,purchases,category,total_purchases
0,orange,2,fruit,2
1,apple,8,fruit,10
2,leek,2,vegetable,2
3,cabbage,9,vegetable,11
4,lettuce,10,vegetable,21
5,kale,23,vegetable,44


Suma skumulowana - obliczana jest w odniesieniu do kolejności określonej za pomocą Order By.

Analiza zaczyna się 2 wiersze przed bieżącym wierszem partycji

In [ ]:
%%bigquery --project produkcja-instore
WITH Produce AS
 (SELECT 'kale' as item, 23 as purchases, 'vegetable' as category
  UNION ALL SELECT 'orange', 2, 'fruit'
  UNION ALL SELECT 'cabbage', 9, 'vegetable'
  UNION ALL SELECT 'apple', 8, 'fruit'
  UNION ALL SELECT 'leek', 2, 'vegetable'
  UNION ALL SELECT 'lettuce', 10, 'vegetable')

SELECT item, purchases, category, 
sum(purchases) over (order by purchases 
ROWS BETWEEN UNBOUNDED PRECEDING AND 2 PRECEDING) total_purchases
FROM produce

,item,purchases,category,total_purchases
0,orange,2,fruit,NaN
1,leek,2,vegetable,NaN
2,apple,8,fruit,2.0
3,cabbage,9,vegetable,4.0
4,lettuce,10,vegetable,12.0
5,kale,23,vegetable,21.0


Średnia ruchoma - Dolna granica to 1 wiersz przed bieżącym wierszem. Górna granica to 1 wiersz za bieżącym wierszem.

In [ ]:
%%bigquery --project produkcja-instore
WITH Produce AS
 (SELECT 'kale' as item, 23 as purchases, 'vegetable' as category
  UNION ALL SELECT 'orange', 2, 'fruit'
  UNION ALL SELECT 'cabbage', 9, 'vegetable'
  UNION ALL SELECT 'apple', 8, 'fruit'
  UNION ALL SELECT 'leek', 2, 'vegetable'
  UNION ALL SELECT 'lettuce', 10, 'vegetable')

SELECT item, purchases, category, 
AVG(purchases) over (order by purchases 
ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) avg_purchases

FROM Produce

,item,purchases,category,avg_purchases
0,orange,2,fruit,2.000000
1,leek,2,vegetable,4.000000
2,apple,8,fruit,6.333333
3,cabbage,9,vegetable,9.000000
4,lettuce,10,vegetable,14.000000
5,kale,23,vegetable,16.500000


Rank()

In [ ]:
%%bigquery --project produkcja-instore
WITH Employees AS
 (SELECT 'Isabella' as name, 2 as department, DATE(1997, 09, 28) as start_date
  UNION ALL SELECT 'Anthony', 1, DATE(1995, 11, 29)
  UNION ALL SELECT 'Daniel', 2, DATE(2004, 06, 24)
  UNION ALL SELECT 'Andrew', 1, DATE(1999, 01, 23)
  UNION ALL SELECT 'Jacob', 1, DATE(1990, 07, 11)
  UNION ALL SELECT 'Jose', 2, DATE(2013, 03, 17))

SELECT name, department, start_date,
rank() over (partition by department order by start_date) rank
FROM Employees

,name,department,start_date,rank
0,Jacob,1,1990-07-11,1
1,Anthony,1,1995-11-29,2
2,Andrew,1,1999-01-23,3
3,Isabella,2,1997-09-28,1
4,Daniel,2,2004-06-24,2
5,Jose,2,2013-03-17,3
